### Step-1: Preliminary setup

In [1]:
#Import gensim library
import gensim
from gensim.models import Word2Vec
from nltk.corpus import PlaintextCorpusReader
import nltk
import pandas as pd
import numpy as np

### Step-2: Load Dataset and Train Word2Vec model

In [2]:
#Load reviews files from the following cities : Chicago, San Francisco, New York City
tokensents_lst=[]
cities = ["chicago", "san-francisco","new-york-city"]
for city in cities:
    print("Loaded City:",city)
    corpus_root = 'OpinRankDataset/hotels/'+city #Root directory
    #Reading all the Corpus in the root directory
    filelists = PlaintextCorpusReader(corpus_root, '.*', encoding='latin-1') 
    tokensents = filelists.sents() # sentence-splitted and word-tokenized list of sentences
    tokensents_lst += list(tokensents) #Concatente all the sentence tokens into one list
#Train the Word2Vec Model
model = Word2Vec(tokensents_lst)    

Loaded City: chicago
Loaded City: san-francisco
Loaded City: new-york-city


### Step-3: Obtaining Feature Words

In [3]:
#Embeding for "service" word
#print(model.wv['Rooms'])

In [4]:
#Similar word vector as the word "service".
#HW5: 'room' (top 50), 'rooms' (top 50), 'Room' (top 20) and 'Rooms' (top 20)

#Initialize the list of feature
feature_lst= []
for word in ["room","rooms", "Room", "Rooms"]:
    if word[0]=="r": #Find top 50 
        word_lst = list(model.most_similar(word, topn=50))
        for i in range(len(word_lst)):
            #print(word_lst[i][0])
            feature_lst.append(word_lst[i][0])
    else:
        word_lst = list(model.most_similar(word, topn=20))
        for i in range(len(word_lst)):
            #print(word_lst[i][0])
            feature_lst.append(word_lst[i][0])
print(set(feature_lst))            
feature_lst = list(set(feature_lst))            

{'window', 'deceptively', 'cupboard', 'sink', 'Bathrooms', 'Towels', 'rooms', 'bathrobe', 'buildings', 'lobbies', 'amenities', 'bedrooms', 'impressively', 'Amenities', 'televisions', 'Rates', 'Rooms', 'cockroach', 'balcony', 'ammenities', 'sofabed', 'refrigerator', 'ones', 'Mine', 'Full', 'guestrooms', 'guestroom', 'livingroom', 'Elevators', 'Beds', 'Linens', 'tub', 'units', 'queen', 'mattresses', 'TVs', 'Bedrooms', 'Portions', 'rooom', 'flatscreens', 'kitchens', 'portable', 'Pillows', 'Breakfasts', 'windows', 'unheated', 'box', 'suites', 'hotels', 'dresser', 'accommodations', 'robe', 'table', 'blanket', 'beds', 'spaces', 'armoire', 'Bedroom', 'bed', 'suitcase', 'plug', 'Gym', 'bathtub', 'Pool', 'backpack', 'washrooms', 'nightstand', 'Food', 'Bathroom', 'towel', 'furnishings', 'Ours', 'mirror', 'toilets', 'Maid', 'balconies', 'terraces', 'Room', 'closets', 'LR', 'king', 'carpets', 'crib', 'baths', 'alcove', 'apartments', 'Hallways', 'wardrobe', 'cot', 'washroom', 'lifts', 'suite', 'clo

/Users/KevQuant/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/Users/KevQuant/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]


### Step-4: hand-write some patterns

In [5]:
#regex grammar
grammar = r"""
JN:{<RB|JJ.*>}
NP:{<DT|PRP\$>?<RB>*<JN>+<NN.*>}
   
clause1:{<DT|PRP/$>*<NN.*>+<VB.*>+<JN>+<JJ.*>*} |
        {<DT|PRP/$>*<NN.*>+<VB.*>+<JN>}
        
clause2:{<clause1>+<,|CC>+<JN|NN.*>+<JN|NN.*>*} |      
        {<clause1>+<,|CC>+<clause1>+}
        
clause3:{<NN.*|PRP.*>+<IN|VB.><DT>*<JN|NN.*>+}        
"""

### Step-5: Extract and Parse all reviews in the city "Chicago"

In [7]:
#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
##################################
test_root = 'OpinRankDataset/hotels/chicago/'
testfile= test_root  + "usa_illinois_chicago_hard_rock_hotel_chicago"
f= open(testfile, "r",encoding='latin-1')
text = f.read()
sentences = nltk.sent_tokenize(text)
sentlist=[nltk.word_tokenize(sent) for sent in sentences]
###################################

#Extract reviews segments which matching the patterns in Step-4
#by using Regex pattern parser
review_lst=[]
for sent in sentlist:
    tree = cp.parse(nltk.pos_tag(sent))
    for subtree in tree.subtrees():
        if subtree.label() =="NP":
            print("NP")
            print(subtree.leaves())
        if subtree.label() =="clause1":
            print("clause1")
            print(subtree.leaves())
            review_lst.append(subtree.leaves())
        if subtree.label() =="clause2":
            print("clause2")
            print(subtree.leaves())
            review_lst.append(subtree.leaves())
        if subtree.label() =="clause3":
            print("clause3")
            print(subtree.leaves())
            review_lst.append(subtree.leaves())

#Filtering out all the segments which do not have feature words              
review_lst2=[]
for line in review_lst:
    for item in line:
        if item[0] in feature_lst:
            review_lst2.append(line)
            continue

NP
[('Fantastic', 'JJ'), ('Experience', 'NNP')]
NP
[('a', 'DT'), ('sleek', 'JJ'), ('exterior', 'NN')]
NP
[('cool', 'JJ'), ('vibe', 'NN')]
NP
[('sooo', 'JJ'), ('cool', 'NN')]
clause3
[('lots', 'NNS'), ('of', 'IN'), ('mirrors', 'NNS')]
NP
[('a', 'DT'), ('wonderfully', 'RB'), ('comfortable', 'JJ'), ('bed', 'NN')]
NP
[('great', 'JJ'), ('bathroom', 'NN')]
clause3
[('bit', 'NN'), ('of', 'IN'), ('a', 'DT'), ('view', 'NN')]
clause3
[('view', 'NN'), ('of', 'IN'), ('Michigan', 'NNP'), ('Avenue', 'NNP')]
NP
[('downtown', 'JJ'), ('Chicago', 'NNP')]
clause3
[('It', 'PRP'), ('is', 'VBZ'), ('absolutely', 'RB'), ('dead-center', 'JJ')]
clause3
[('block', 'NN'), ('from', 'IN'), ('the', 'DT'), ('river', 'NN')]
clause3
[('we', 'PRP'), ('were', 'VBD'), ('able', 'JJ')]
clause3
[('Festival', 'NNP'), ('of', 'IN'), ('Lights', 'NNP'), ('parade', 'NN')]
NP
[('that', 'DT'), ('great', 'JJ'), ('window', 'NN')]
clause3
[('It', 'PRP'), ("'s", 'VBZ'), ('a', 'DT'), ('block', 'NN')]
clause3
[('couple', 'NN'), ('of', 'IN

[('ago', 'RB'), ('AND', 'NNP')]
clause1
[('the', 'DT'), ('concierge', 'NN'), ('desk', 'NN'), ('was', 'VBD'), ('so', 'RB'), ('helpful', 'JJ')]
NP
[('an', 'DT'), ('AMAZING', 'JJ'), ('restaurant', 'NN')]
clause3
[('I', 'PRP'), ('love', 'VBP'), ('the', 'DT'), ('music', 'NN')]
NP
[('overall', 'JJ'), ('feel', 'NN')]
clause3
[('fact', 'NN'), ('that', 'IN'), ('the', 'DT'), ('Hard', 'NNP'), ('Rock', 'NNP'), ('Hotel', 'NNP')]
NP
[('my', 'PRP$'), ('next', 'JJ'), ('stay', 'NN')]
NP
[('great', 'JJ'), ('people', 'NNS')]
NP
[('great', 'JJ'), ('location', 'NN')]
clause2
[('The', 'DT'), ('staff', 'NN'), ('was', 'VBD'), ('incredible', 'JJ'), ('and', 'CC'), ('the', 'DT'), ('rooms', 'NNS'), ('were', 'VBD'), ('great', 'JJ')]
clause1
[('The', 'DT'), ('staff', 'NN'), ('was', 'VBD'), ('incredible', 'JJ')]
clause1
[('the', 'DT'), ('rooms', 'NNS'), ('were', 'VBD'), ('great', 'JJ')]
clause2
[('The', 'DT'), ('location', 'NN'), ('is', 'VBZ'), ('pretty', 'JJ'), ('prime', 'JJ'), (',', ','), ('the', 'DT'), ('Metra', 

clause3
[('it', 'PRP'), ('did', 'VBD'), ("n't", 'RB')]
clause1
[('something', 'NN'), ('was', 'VBD'), ('wrong', 'JJ')]
NP
[('a', 'DT'), ('consistent', 'JJ'), ('noise', 'NN')]
NP
[('every', 'DT'), ('few', 'JJ'), ('seconds', 'NNS')]
clause3
[('I', 'PRP'), ('called', 'VBD'), ('the', 'DT'), ('front', 'NN'), ('desk', 'NN')]
clause1
[('the', 'DT'), ('engineer', 'NN'), ('came', 'VBD'), ('up', 'RB')]
clause3
[('staff', 'NN'), ('member', 'NN'), ('at', 'IN'), ('the', 'DT'), ('front', 'NN'), ('desk', 'NN')]
clause1
[('room', 'NN'), ('was', 'VBD'), ('close', 'JJ')]
NP
[('a', 'DT'), ('mechanical', 'JJ'), ('room', 'NN')]
clause3
[('I', 'PRP'), ('was', 'VBD'), ('not', 'RB'), ('happy', 'JJ')]
NP
[('an', 'DT'), ('awesome', 'JJ'), ('job', 'NN')]
NP
[('a', 'DT'), ('beautiful', 'JJ'), ('Tower', 'NNP')]
NP
[('several', 'JJ'), ('passes', 'NNS')]
NP
[('the', 'DT'), ('beautiful', 'JJ'), ('breakfast', 'NN')]
clause3
[('buffet', 'NN'), ('at', 'IN'), ('the', 'DT'), ('China', 'NNP'), ('Grill', 'NNP'), ('restaurant

clause3
[('It', 'PRP'), ('is', 'VBZ'), ('only', 'RB')]
NP
[('a', 'DT'), ('short', 'JJ'), ('block', 'NN')]
clause3
[('tons', 'NNS'), ('of', 'IN'), ('restaurants', 'NNS')]
clause3
[('We', 'PRP'), ('were', 'VBD'), ('never', 'RB')]
NP
[('a', 'DT'), ('great', 'JJ'), ('weekend', 'NN')]
NP
[('a', 'DT'), ('few', 'JJ'), ('quirks', 'NNS')]
clause3
[('I', 'PRP'), ('was', 'VBD'), ('a', 'DT'), ('little', 'JJ'), ('concerned', 'JJ')]
NP
[('the', 'DT'), ('negative', 'JJ'), ('reviews', 'NNS')]
NP
[('the', 'DT'), ('most', 'JJS'), ('part', 'NN')]
NP
[('overblown.The', 'JJ'), ('hotel', 'NN')]
NP
[('an', 'DT'), ('historic', 'JJ'), ('Art', 'NNP')]
NP
[('a', 'DT'), ('fabulous', 'JJ'), ('location', 'NN')]
clause3
[('right', 'NN'), ('on', 'IN'), ('Michigan', 'NNP'), ('Ave', 'NNP')]
clause1
[('The', 'DT'), ('room', 'NN'), ('is', 'VBZ'), ('very', 'RB'), ('nice', 'JJ')]
NP
[('a', 'DT'), ('very', 'RB'), ('comfortable', 'JJ'), ('bed', 'NN')]
NP
[('a', 'DT'), ('large', 'JJ'), ('flat', 'JJ'), ('screen', 'JJ'), ('tv',

clause3
[('it', 'PRP'), ('was', 'VBD'), ('exactly', 'RB')]
clause3
[('it', 'PRP'), ('on', 'IN'), ('the', 'DT'), ('website', 'NN')]
clause1
[('heat', 'NN'), ('is', 'VBZ'), ('very', 'RB')]
NP
[('not', 'RB'), ('much', 'JJ'), ('effort', 'NN')]
clause3
[('Great', 'NNP'), ('Value', 'NNP'), ('For', 'IN'), ('Single', 'NNP'), ('Fathers', 'NNS')]
NP
[('here', 'RB'), ('August', 'NNP')]
NP
[('my', 'PRP$'), ('sixteen', 'JJ'), ('year', 'NN')]
NP
[('old', 'JJ'), ('son', 'NN')]
clause1
[('The', 'DT'), ('price', 'NN'), ('was', 'VBD'), ('very', 'RB'), ('reasonable', 'JJ')]
clause1
[('a', 'DT'), ('hotel', 'NN'), ('located', 'VBN'), ('right', 'RB')]
clause1
[('concierge', 'NN'), ('service', 'NN'), ('was', 'VBD'), ('excellent', 'JJ')]
NP
[('fun', 'JJ'), ('activities', 'NNS')]
NP
[('a', 'DT'), ('nice', 'JJ'), ('touch', 'NN')]
NP
[('single', 'JJ'), ('fathers', 'NNS')]
NP
[('affordable', 'JJ'), ('vacation', 'NN')]
clause3
[('Hard', 'NNP'), ('Rock', 'NNP'), ('Hotel', 'NNP'), ('in', 'IN'), ('Chicago', 'NNP'), (

...

clause2
[('Beds', 'NNS'), ('are', 'VBP'), ('nice', 'JJ'), ('and', 'CC'), ('comfy', 'NN')]
clause1
[('Beds', 'NNS'), ('are', 'VBP'), ('nice', 'JJ')]
NP
[('fluffy', 'JJ'), ('pillows', 'NNS')]
NP
[('covers.Some', 'JJ'), ('great', 'JJ'), ('perks', 'NNS')]
clause3
[('-Each', 'NN'), ('room', 'NN'), ('has', 'VBZ'), ('a', 'DT'), ('disc', 'NN'), ('changer', 'NN')]
clause3
[('concierge', 'NN'), ('has', 'VBZ'), ('a', 'DT'), ('ton', 'NN')]
NP
[('great', 'JJ'), ('CD', 'NN')]
clause3
[('Taste', 'NN'), ('of', 'IN'), ('Chicago', 'NNP')]
clause3
[('restaurant', 'NN'), ('called', 'VBN'), ('Giordano', 'NNP')]
NP
[('the', 'DT'), ('best', 'JJS'), ('stuffed', 'JJ'), ('pizza', 'NN')]
clause1
[('i', 'NN'), ('have', 'VBP'), ('ever', 'RB'), ('tasted.-Even', 'RB')]
NP
[('the', 'DT'), ('regular', 'JJ'), ('rooms', 'NNS')]
NP
[('the', 'DT'), ('second', 'JJ'), ('day', 'NN')]
clause3
[('we', 'PRP'), ('got', 'VBD'), ('a', 'DT'), ('FANTASTIC', 'NNP'), ('room', 'NN')]
NP
[('beautiful', 'JJ'), ('view', 'NN')]
clause3
[('

clause3
[('Hard', 'NNP'), ('Rock', 'NNP'), ('Hotel', 'NNP'), ('in', 'IN'), ('Chicago', 'NNP')]
NP
[('several', 'JJ'), ('times', 'NNS')]
NP
[('a', 'DT'), ('great', 'JJ'), ('place', 'NN')]
clause2
[('The', 'DT'), ('staff', 'NN'), ('is', 'VBZ'), ('extremely', 'RB'), ('friendly', 'JJ'), ('and', 'CC'), ('helpful', 'JJ')]
clause1
[('The', 'DT'), ('staff', 'NN'), ('is', 'VBZ'), ('extremely', 'RB'), ('friendly', 'JJ')]
clause2
[('the', 'DT'), ('rooms', 'NNS'), ('are', 'VBP'), ('immaculate', 'JJ'), ('and', 'CC'), ('beautifully', 'RB')]
clause1
[('the', 'DT'), ('rooms', 'NNS'), ('are', 'VBP'), ('immaculate', 'JJ')]
NP
[('this', 'DT'), ('past', 'JJ'), ('January', 'NNP')]
NP
[('a', 'DT'), ('wonderful', 'JJ'), ('treat', 'NN')]
clause1
[('degree', 'NN'), ('views', 'NNS'), ('looking', 'VBG'), ('up', 'RB')]
NP
[('down', 'RB'), ('Michigan', 'NNP')]
clause3
[('Avenue', 'NNP'), ('with', 'IN'), ('views', 'NNS')]
clause2
[('the', 'DT'), ('amenities', 'NNS'), ('are', 'VBP'), ('first-class', 'JJ'), ('and', '

clause2
[('The', 'DT'), ('Hard', 'NNP'), ('Rock', 'NNP'), ('Hotel', 'NNP'), ('is', 'VBZ'), ('not', 'RB'), ('very', 'RB'), ('warm', 'JJ'), ('or', 'CC'), ('welcoming', 'NN')]
clause1
[('The', 'DT'), ('Hard', 'NNP'), ('Rock', 'NNP'), ('Hotel', 'NNP'), ('is', 'VBZ'), ('not', 'RB'), ('very', 'RB'), ('warm', 'JJ')]
NP
[('the', 'DT'), ('front', 'JJ'), ('door', 'NN')]
clause3
[('my', 'PRP$'), ('reservation', 'NN'), ('because', 'IN'), ('each', 'DT'), ('time', 'NN')]
clause3
[('I', 'PRP'), ('got', 'VBD'), ('someone', 'NN')]
NP
[('additional', 'JJ'), ('questions', 'NNS')]
NP
[('an', 'DT'), ('obvious', 'JJ'), ('interruption', 'NN')]
clause3
[('IRRITATING', 'NN'), ('cliche', 'NN'), ('with', 'IN'), ('nothing', 'NN')]
NP
[('borderline', 'JJ'), ('rudeness', 'NN')]
clause3
[('Comfort', 'NNP'), ('Inn', 'NNP'), ('on', 'IN'), ('Ohio', 'NNP')]
NP
[('a', 'DT'), ('safe', 'JJ'), ('pleasant', 'JJ'), ('place', 'NN')]
clause3
[('rest', 'NN'), ('of', 'IN'), ('the', 'DT'), ('time', 'NN')]
clause3
[('Stay', 'NNP'),

clause3
[('manager', 'NN'), ('at', 'IN'), ('checkout', 'NN')]
NP
[('the', 'DT'), ('continental', 'JJ'), ('breakfast', 'NN')]
clause3
[('you', 'PRP'), ('consumed', 'VBD'), ('the', 'DT'), ('breakfast', 'NN')]
clause3
[('Marriott', 'NNP'), ('cost', 'NN'), ('about', 'IN'), ('the', 'DT'), ('same', 'JJ')]
NP
[('much', 'JJ'), ('classier', 'JJR'), ('hotels', 'NNS')]
NP
[('last', 'JJ'), ('year', 'NN')]
NP
[('a', 'DT'), ('wonferful', 'JJ'), ('time', 'NN')]
clause1
[('The', 'DT'), ('rooms', 'NNS'), ('are', 'VBP'), ('nicely', 'RB')]
clause1
[('the', 'DT'), ('beds', 'NNS'), ('are', 'VBP'), ('very', 'RB'), ('comfortable', 'JJ')]
clause3
[('hotel', 'NN'), ('provides', 'VBZ'), ('a', 'DT'), ('bathrobe', 'NN')]
NP
[('a', 'DT'), ('long', 'JJ'), ('day', 'NN')]
clause3
[('fun', 'NN'), ('in', 'IN'), ('the', 'DT'), ('Windy', 'NNP'), ('City', 'NNP')]
NP
[('The', 'DT'), ('flat', 'JJ'), ('panel', 'NN')]
clause3
[('tv', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('plus', 'JJ')]
clause3
[('surround', 'NN'), ('system', 'N

clause3
[('velvet', 'NN'), ('ropes', 'VBZ'), ('Me', 'NNP')]
clause1
[('Dad', 'NNP'), ('went', 'VBD'), ('up', 'RB')]
clause3
[('city', 'NN'), ('for', 'IN'), ('a', 'DT'), ('weekend', 'NN')]
NP
[('an', 'DT'), ('excellent', 'JJ'), ('show', 'NN')]
NP
[('i', 'JJ'), ('night', 'NN')]
clause3
[('rooms', 'NNS'), ('were', 'VBD'), ('a', 'DT'), ('bit', 'NN'), ('small', 'JJ')]
clause1
[('The', 'DT'), ('service', 'NN'), ('was', 'VBD'), ('very', 'RB'), ('good', 'JJ')]
clause3
[('bit', 'NN'), ('of', 'IN'), ('an', 'DT'), ('attude', 'NN')]
NP
[('a', 'DT'), ('chicago', 'JJ'), ('thing', 'NN')]
NP
[('a', 'DT'), ('cool', 'JJ'), ('place', 'NN')]
NP
[('much', 'JJ'), ('cooler', 'NN')]
NP
[('a', 'DT'), ('free', 'JJ'), ('compulation', 'NN')]
clause3
[('I', 'PRP'), ('think', 'VBP'), ('alot', 'NN')]
NP
[('amazing', 'JJ'), ('part', 'NN')]
clause3
[('Hard', 'NNP'), ('rock', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('Concierge', 'NNP')]
clause1
[('This', 'DT'), ('guy', 'NN'), ('is', 'VBZ'), ('CONNECTED', 'JJ')]
clause3
[(

### Step-6: Polarity Score

##### Given Functions

In [8]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

def to_wntag(pos):
    """
    Given a Penn tag, returns the corresponding Wordnet pos tag.
    Note that WordNet only contain nouns/verbs/adjectives/adverbs.
    """
    ch = pos[0]
    if ch == 'N':
        return wn.NOUN #noun
    elif ch == 'V':
        return wn.VERB # verb
    elif ch == 'J':
        return wn.ADJ # adjective
    elif ch == 'R':
        return wn.ADV # adverb
    else:
        return None

def wn_lemmatize(token, wntag):
    """
    Applies WordNet morpher (which requires a pos) and 
    returns the morphological lemma form, or None.
    """
    return wn.morphy(token, wntag)

In [ ]:
#review_lst2= [[('The', 'DT'), ('roOm', 'NNS'), ('is', 'VBP'), ('Not', 'RB'), ('HUGE', 'JJ')],
 #           [('the', 'DT'), ('rooms', 'NNS'), ('are', 'VBP'), ('very', 'RB'), ('Small', 'JJ')],
  #         [('weird', 'JJ'), ('Windows', 'NNS')],
   #       [('The', 'DT'), ('Beds', 'NNS'), ('were', 'VBD'), ('Very', 'RB'), ('good', 'JJ')],
    #     [('the', 'DT'), ('Elevators', 'NNS'), ('are', 'VBZ'), ('Pretty', 'JJ')],
     #   [('the', 'DT'), ('Elevator', 'NNS'), ('is', 'VBZ'), ('Pretty', 'JJ')],
      #       [("Awesome", 'NNP'),('Rooms', 'NNS'),('Comfy', 'VBP')]]
#review_lst2 = [[('amenities', 'NNS'), ('were', 'VBD'), ('excellent', 'JJ'), ]]

In [9]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
df_lst = [0 for item in range(len(review_lst2))]
df_lst2 = []
df_lst1 = []

cnt=0
for tups in review_lst2:
    lst=[]
    tot_pscore=0
    word_cnt=0
    for i in range(len(tups)):
        tups[i]=list(tups[i]) #Convert the list back to tuples
        w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
        tag=tups[i][1] #Assign the tag from the 2nd element of tuples
        
        #If the word is a negative contraction suffix ('negcontractions'), 
        #change it to the word: "not"
        if w in negcontractions:
            w="not"

        # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
        wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
        if wntag is not None and (w not in stopwords or w in negwords): 
            print(w)
            tups[i][0] = w 
            word_cnt+=1
            lemma = wn_lemmatize(w, wntag) #lemmatize token

            if lemma is None:
                tups[i]=tuple(tups[i])
                continue

            synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

            if not synsets:
                tups[i]=tuple(tups[i])
                continue

            # take the first sense
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
            pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
            tot_pscore+=pscore
            
            #Convert the list back to tuples
            tups[i]=tuple(tups[i])

        else:
            tups[i][0] = w 
            tups[i]=tuple(tups[i])
            
    df_lst1.append(tot_pscore/word_cnt)
    df_lst2.append(tups)
    cnt+=1
    
    
#Concatnate the tuples and Polarity Score in a DataFrame
#df = pd.DataFrame({ i:pd.Series(value) for i, value in enumerate(df_lst) })
df1= pd.DataFrame(df_lst1, columns= ["Polarity_Score"] )
df2= pd.DataFrame(df_lst2)
df = pd.concat((df1,df2), axis=1)
df = df.sort_values(by=["Polarity_Score",0,1,2,3,4,5,6,7,8,9,10], 
                    ascending=[False, True, True,True, True,True,True, True,True, True,True,True])

bathroom
smaller
rooms
nice
rooms
imaculately
clean
nicely
rooms
imaculately
clean
room
incredible
rooms
great
showers
windows
bed
abd
pillows
extremely
comfortable
toothpaste
room
rooms
nice
clean
rooms
nice
bathrooms
great
staff
incredible
rooms
great
rooms
great
room
great
bed
comfortable
room
great
bed
comfortable
room
great
bed
comfortable
television
lcd
rooms
couples
room
beds
room
beds
room
bathrooms
not
much
better
got
rooms
choice
rooms
took
room
plethora
room
rooms
dark
bathroom
clean
roomy
bathroom
clean
corner
room
room
close
room
room
not
clean
lotion
bathroom
elevators
saturday
morning
price
tag
room
room
worn
bed
immaculate
bathroom
nicely
mirror
large
enough
rooms
clean
order
room
service
breakfast
elevators
significantly
rooms
quiet
rooms
much
bigger
bathrooms
bit
tight
room
clean
housekeeping
seemed
meticulous
room
clean
room
nice
room
packages
room
clean
room
service
expensive
hotel
room
huge
nice
hotel
room
huge
rooms
small
room
scratches
room
bit
drab
room
great
vi

room
room
nice
comfortable
room
nice
hole
passenger
window
room
really
bad
room
service
took
forever
room
small
beds
really
comfy
rooms
tastefully
issues
room
service
room
clean
bed
extremely
comfortable
bathroom
decent
room
service
room
entirely
gray
room
room
seemed
little
warm
room
ready
rooms
nicely
beds
comfortable
hotel
provides
bathrobe
surround
system
room
lighting
room
rooms
best
room
service
done
twice
rooms
dark
dirty
rooms
dark
rooms
clean
trendy
rooms
clean
booked
room
bed
amazing
hotel
bed
rooms
suprisingly
bigger
room
small
room
great
room
service
way
cheaper
rooms
bit
small
mainstreem
advertised
ones


In [10]:
pre_lst=[]
new_lst=[]
cnt=0
with open("XXXX-rooms.txt", "w") as f:
    for row in range(0,df.shape[0]):
        new_lst = list(df.iloc[row])
        new_lst = [item  for item in new_lst if item is not None]
        #print(new_lst)

        if pre_lst == new_lst:
            continue
            
        for item in df.iloc[row]:
            if item is not None:
                try:
                    if type(float(item)) == float:
                        #print(item, end=", [")
                        f.write("{0}, [".format(item))
                        continue
                except:
                    pass
                #print(item, end=",")
                f.write("{0}, ".format(item))

            else:
                #print("]",end="\n")
                f.write("]\n")
                cnt+=1
                break
        pre_lst = new_lst
f.close()
print(cnt)    

568


In [ ]:
df

########################END

In [ ]:
#sent = nltk.pos_tag(nltk.word_tokenize("She girl let down her long hair"))
sent = nltk.pos_tag(nltk.word_tokenize("quick"))
print(sent)

#regex grammar
grammar = r"""
NP:{<DT|PRP\$>?<JJ>*<NN>} 
FOO:{<NP><VB.><JJ>}"""
#"NP:{DT|PRP\$>?<JJ>*<NN>}" #, FOO: {<NP><VB.><JJ>}"

#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)



In [ ]:
#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)


In [ ]:
for subtree in tree.subtrees():
    if subtree.label() =="NP":
        print(subtree.leaves())

In [ ]:
for sent in sentlist:
    cp.parse(nltk.pos_tag(sent))
    for subtree in tree.subtrees():
        if subtree.label() =="NP":
            print(subtree.leaves())